<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Visualization-in-rviz" data-toc-modified-id="Visualization-in-rviz-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Visualization in rviz</a></span></li><li><span><a href="#Simultaneous-Localization-and-Mapping-(SLAM)" data-toc-modified-id="Simultaneous-Localization-and-Mapping-(SLAM)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Simultaneous Localization and Mapping (SLAM)</a></span><ul class="toc-item"><li><span><a href="#Gmapping" data-toc-modified-id="Gmapping-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Gmapping</a></span></li><li><span><a href="#Google-cartographer" data-toc-modified-id="Google-cartographer-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Google cartographer</a></span></li><li><span><a href="#tinySLAM" data-toc-modified-id="tinySLAM-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>tinySLAM</a></span></li></ul></li><li><span><a href="#Navigation-stack" data-toc-modified-id="Navigation-stack-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Navigation stack</a></span></li></ul></div>

# Hands-on with the Turtlebot 3

This final part of the workshop is focused on gaining practical experience with ROS. To this end, we will make use of the new, low-cost robot platform [Turtlebot 3](http://www.turtlebot.com/). This is the flagship robot of ROS. We use the third iteration of this platform, more specifically the burger model. It is comprised of a few main components:

- **LIDAR** : on the very top of the Turlebot, you will find a 360° laser scanner. This sensor transmits a rotating laser, which reflects of nearby obstacles. By using the reflection time, distance to the obstacles can be obtained. These distances can be used to build a map of the environment.
- **Raspberry Pi**: underneath the LIDAR you will find a Raspberry Pi 3. This credit card-sized computer runs a special version of ubuntu linux. Since its introduction a few years ago, the Raspberry Pi has gained incredible popularity among hobbyists and researchers due to its small size, low cost and power use. It is most usefull in applications that need a computer, but not a huge amount of processing power. On the Turtlebot, its job is to read data from the sensors and communicate this with a 'master' pc, which does the actual computations.
- **OpenCR**: just above the battery and the motors, you will find the hardware control board. It contains an onboard IMU with a three axis accelerometer, gyroscope and magnetometer. Its main function is to connect the raspberry pi to the motors and the sensors.

![turtlebot3_burger](http://emanual.robotis.com/assets/images/platform/turtlebot3/hardware_setup/turtlebot3_burger_components.png)

In the following sections, you will learn how to visualize data published on ROS topics with rviz. Next, we will discuss rosbag, a tool that allows you to record data from ROS topics and replay it later (without a robot). Finally, you will use the navigation stack to let your robot drive autonomously.

## Visualization in rviz

rviz is a 3D visualization tool for ROS. It is included in a full desktop install of ROS, but can of course be installed separately as well. rviz can display robot positions (with a complete robot model if it is provided in a URDF file), laser scans, point clouds and much more information that is being published by ROS nodes. To start rviz, a roscore needs to be running, so open a new terminal (either from the jupyter server tab by selecting 'new' -> 'terminal', or with the keyboard shortcut `Ctrl+Alt+T`) and run `roscore`. 

In a separate terminal, you can start rviz:

In [ ]:
!rviz

You can use the left mouse button to rotate the camera, the right mouse button to zoom and the middle mouse button 
to pan. On the left hand side of the window, the display options are listed. The global options are listed at the top. Parameters like background color and frame rate can be adjusted here. There is also an option called 'Fixed Frame', which referes to the static coordinate frame relative to which all other information is referenced. By default, the frame 'odom' is selected, but 'base_footprint', 'world' and 'map' are also often used. The fixed frame should not be moving (e.g. it should not be attached to a mobile robot), doing so might result in erroneous visualization. 

![](images/rviz_display_options.png)

Below the global options, you can adjust the settings of the visualization grid, such as the cell size and the color. At the bottom left of the rviz window, the 'Add' button allows you to visualize additional topics. The topic that is arguably the most usefull to visualize in rviz, is the `/tf` topic. tf is an abbreviation for transfer frame, and contains information about the relative location of all coordinate frames in the system. Eventually, all frames should be directly or indirectly referenced relative to the fixed frame, thus building a so called 'tf-tree'.

By selecting the `tf` message type, an additional display option will appear. You can then set parameters like the frame timeout and the update interval.

You can add additional display options in a similar manner. You can even have two display options of the same type (for example, 2 camera displays). This option only makes sense when the data being displayed originates from different topics though.

Once you have added the desired display options, you can save your configuration as a .rviz file. Doing so will allow you to have the exact same visualization window in the future (and thus prevents having to add the same topics over and over again). To save your rviz file:

1. Select the 'file' option in the window bar of the rviz panel. 
2. Select 'save config as'. 
3. Choose an appropriate name and location for your file. Select the default location (/home/ROS-workshop/.rviz) and 'example' as a filename to follow along with the rest of the tutorial.

You can now safely close rviz. To open the same configuration file in rviz, you can either type the ```rviz``` command in a terminal, then navigate to the file be selecting 'file' and then 'open'. Alternatively, you can pass the configuration file as an argument when opening rviz:

In [ ]:
!rviz -d $HOME/.rviz/example.rviz

The argument after ```-d``` should be the path to the rviz file. If everything works correctly, you should see the display options you added earlier in stead of the default window.

## Simultaneous Localization and Mapping (SLAM)

The SLAM (Simultaneous Localization and Mapping) problem is the problem of building a map of the environment while simultaneously determining the robot’s position relative to this map. This problem is also referred to as Concurrent Mapping and Localization (CML). A mobile robot moves through an unknown environment, starting at a known location (often given by coordinates {0,0,0}). Its motion is uncertain due to the robot’s model and sensor noise.
As it roams, the robot can detect the environment using a form of observation measurement. To compute the map and the robot’s location within this map, the probability of accuracy of the data must be included. 

SLAM has received considerable attention since the 1990's. Several approaches have been proposed such as Kalman and Particle filter SLAM. In the following sections, we will experiment with 3 different SLAM algorithms available in ROS. Finally, we will use the generated maps for autonomous navigation in a real-life environment.

### Gmapping
Gmapping is arguably the most popular software implementation of SLAM. It employs a Rao-Blackwellized particle filter (RBPF) and builds a 2D map based on laser scan data. The code and documentation are available on the [OpenSLAM website](http://openslam.org/gmapping.html). It is very easy to integrate into ROS, and can even be run on a low cost robot-platform like our Turtlebots.

We will now move on to a more practical example of SLAM in ROS by building map with gmapping and a Turtlebot 3. The RPi on the robot has sends commands to the motors and reads out the various sensors. It relays this information to a remote pc, which does the actual processing. The RPi's on our robots have been configured to function as an access point, which means that a connection from your computer to the robot can be established just like any other wifi-connection. There is no central connection point, and thus no central point of failure. 

Before we can move on to the actual mapping, a roscore needs to be started on the robot, which will listen to the roscore on your PC in a master/slave configuration. You will have to enter some commands on your PC, which will be denoted by: **[PC]**. Some commands need to be executed on the raspberry pi over SSH, these commands are prefaced by **[RPi]**.

2. Flip the switch at the back of the Turtlebot to turn it on.
2. On the remote **[PC]** (master) connect to the Turtlebot 3 wifi (password = groupRBP).  It can take a few minutes for the robot to show in the available wireless connections. ![](images/TB_wifi.png)
3. Start a `roscore` and leave it running in a seperate terminal
3. Establish an [SSH](https://github.com/RobinAmsters/ROS_workshop/wiki/Linux-basics#ssh) (secure shell) connection:
    1. Open a terminal and type: ssh rosberrypi@192.168.42.1
    2. When prompted for a password, type: *groupRBP*. Unlike most applications, you will not see any symbols while typing the password. However, the keys are still being registered. So type the password and hit enter.
    3. If you are asked to add rosberrypi to the list of known hosts, type 'yes' and press enter
    4. If everything went as expected, the user in the terminal should have changed. All your commands should now be preceded by *'rosberrypi@rosberrypi'*. Any command you now type in this terminal is executed on the raspberry pi. 
4. **[RPi]** Start a roscore on the pi with the following command: ```bringup```. Besides starting a regular roscore, communication with the LIDAR and motors is now initialized as well.
5. To test that everything is working correctly, open a new terminal and type the following command on your own **[PC]**:
```teleop```. You should now be able to drive the Turtlebot around with your WASD keys on your keyboard. You can change your keyboard layout to QWERTY by clicking on the 'BE' symbol in the top left next to the wireless connection symbol. When you are done, end the process with `CTRL+C`. Leave the other two terminals open.

We are now ready to start gmapping. Open a new terminal and run:

In [ ]:
!roslaunch workshop_tutorial turtlebot_gmapping.launch

This will start gmapping and rviz. You can steer the Turtlebot with your keyboard. The map will get updated and shown in rviz as you drive. 

![Gmapping example](images/gmapping_example.png)

Gmapping builds an occupancy grid type map, which means explored terrain is either marked as free space or obstacles. Areas marked in white are considered free space where the robot can drive. Black is used to mark the location of (static) obstacles. Grey areas are unexplored or uncertain areas.

To save the map run:

In [ ]:
!rosrun map_server map_saver -f ~/ROS_workshop/data/map

This will save the map in the subdirectory 'data' of the 'ROS_workshop' directory wth the filename 'map'. The map will be saved in both .pgm and .yaml format. The .pgm file is mostly for visual inspection. These files can be used for autonomous navigation, which we will test in the navigation stack section. 

You also have the option to save all data during a test in a .bag file:

In [ ]:
!roslaunch workshop_tutorial turtlebot_gmapping.launch record:=True

Afterwards the data can be replayed by using the 'play' option. A bagfile should also be specified, so don't forget to change the filename in the command below. You should see the map being built again, except the Turtlebot will not be moving this time. The same options can be used in subsequent mapping sections.

In [ ]:
!roslaunch workshop_tutorial turtlebot_gmapping.launch play:=True bagfile:=2018-03-29-14-30.bag

### Google cartographer
[Cartographer](https://opensource.google.com/projects/cartographer) is an open source SLAM library developed by Google. It can be used to build both 2D and 3D maps by combining information from laser scans, point clouds, IMU's and odometry measurements. During Google's test with backpack mapping, the algorithms achieved real-time mapping and loop closure at a 5 cm resolution. Experimental results and comparisons to other well known approaches show that, in terms of quality, Cartographer is competitive with established techniques.

Cartographer comes with a [ROS API](https://github.com/googlecartographer/cartographer_ros), and can be used more or less out-of-the box with several ROS-enabled robots, including the Turtlebot. You can test it yourself by starting the custom launch file:

In [ ]:
!roslaunch workshop_tutorial turtlebot_cartographer.launch

The map built by cartographer is bit different then the map built by Gmapping, and we have to use a different command to save the map.

In [ ]:
!rosrun occupancy_grid_saver map_listener

This will save the map in your current directory. Make sure to change the nan to 0.0

### tinySLAM

tinySLAM is a SLAM algorithm developed to be as simple as possible. It uses a particle filter based localization subsystem. It uses raw laser range data and odometry to create a grid map and find the position of the robot. A great difference is that the algorithm immediately defines a pose of the robot. Other algorithms using particle filters, usually maintain a probability. With this probability, the algorithm defines multiple points where the robot might be. When more data is available, the program can then decide where the robot actually is and chose one probability. TinySLAM doesn’t do this. It immediately tries to define the highest probability and computes the map from that point. In other words, other SLAM’s with particle filters have multiple maps to overcome uncertainties. TinySLAM has only one map.

Once again, a launch file is available for you to experiment with the algorithm yourself. 

In [ ]:
!roslaunch workshop_tutorial turtlebot_tinySLAM.launch

Like the gmapping launch file, the record and play options can be used to record or play the data in a rosbag file. To save the map from tinySLAM, a costum ROS package is required:

In [ ]:
!rosrun occupancy_grid_saver map_listener

## Navigation stack

The [ROS navigation stack](http://wiki.ros.org/navigation) is a collection of packages that enable autonomous operation for ROS-enabled robots. Sensor data such as odometry and laser scans is taken as an input, and actuator commands are sent to a mobile robot. Succesfull use of the navigation stack depends on a number of hard and software requirements.

Software requirements:
 - The robot must be controlled via a ROS master
 - A [tf](http://wiki.ros.org/tf) transform tree must exist which describes the relation of the robot joints.
 - Sensor data must be published in the right message types.
 - The navigation stack must be configured according to the shape and dynamics of the robot.
 
 Hardware requirements:
- The stack is meant for both differential drive and holonomic wheeled robots only. It assumes that the mobile base is controlled by sending desired velocity commands to achieve in the form of: x velocity, y velocity, theta velocity.
- A planar laser mounted somewhere on the mobile base is required. This laser is used for map building and localization.
- The Navigation Stack was developed on a square robot, so its performance will be best on robots that are nearly square or circular. It does work on robots of arbitrary shapes and sizes, but it may have difficulty with large rectangular robots in narrow spaces like doorways.

A list of robots which use the navigation stack can be found on the [ROS wiki](http://wiki.ros.org/navigation/RobotsUsingNavStack). Fortunately, the Turtlebot 3 is on that list. As such we can use it to get some hands-on experience. Make sure you have a yaml file with the name 'map' in the ROS_workshop/data folder. Additionally, the clock of the Turtlebot and your PC need to be approximately synchronized. Unfortunately the raspberry pi does not have a real time clock, which means that the clock resets every time the pi reboots. Therefore we first need to synchronize the clocks. Open a new terminal and navigate to the ROS_workshop folder:


In [ ]:
!cd ~/ROS_workshop/

Next we will execute a so called 'shell script'. This is simply a set of instructions that run in your terminal one after the other. You can automate a lot of tasks with shell scripts such as copying files, pinging servers, etc.  Our shell script will save the current date, send it to the pi over SSH, and set that as the pi's current date. 

A shell script is run by prefacing the name of the script with './' .

In [ ]:
!./update_clock.sh

You will need to type the password of the raspberrypi (groupRBP) twice during executing. As usual, you will not see any feedback such as stars or dots to indicate how many keys you have pressed. Keys are still being registered, however, so just type the password and press enter. 

Finally, we can start the turtlebot navigation stack launch file:

In [ ]:
!roslaunch workshop_tutorial turtlebot_navigation

You will now see an rviz window similar to the SLAM section. The occupancy grid map is displayed just like before, but some extra information is added. The mean estimated location of the robot is shown as a small robot model. Other pose estimate particles (which are tracked by [AMCL](http://wiki.ros.org/amcl)) are displayed with green dots. The blue/purple/red blobs indicate the location of detected obstacles in the robot frame. 

![navigation_example](images/navigation_example.png)

Most likely the initial position estimate will not be correct, and the detected obstacles will not be aligned with the map. Before starting the navigation, this initial estimate should be corrected. 

1. At the top of the rviz window, click the '2D Pose Estimate button'.
2. Click on the approximate point in the map where the TurtleBot3 is located and drag the cursor to indicate the direction in which it faces.

Repeat this proces if necessary untill the detected obstacles are approximately aligned with the map. Once the Turtlebot initial pose is set, a goal pose can be set in a similar manner with the '2D Nav Goal' button. A trajectory will now be planned, and the robot will start driving. As it moves, the spread of pose estimates (the green dots) will become smaller as the particle filter converges. If the robot encounters an unplanned obstacle (such as people walking around), then it will recompute a new trajectory.

When you are done, you can close all processes with `CTRL+C` (you might need to press is more then once for large launch files such as the navigation file). Before closing the SSH connection (the terminal that starts with rosberrypi@rosberrypi) shut down the pi:

In [ ]:
sudo shutdown now

After about 1 minute, it is safe to turn the robot off with the power switch.